In [ ]:
import io
import grpc
import os
import json
import time
import pravega.grpc_gateway as pravega

In [ ]:
gateway = '10.246.27.131:54672'
scope = 'examples'
stream = 'memory-eater-in'

In [ ]:
pravega_channel = grpc.insecure_channel(gateway)
pravega_client = pravega.grpc.PravegaGatewayStub(pravega_channel)

In [ ]:
pravega_client.DeleteStream(pravega.pb.DeleteStreamRequest(scope=scope, stream=stream))

In [ ]:
pravega_client.CreateStream(pravega.pb.DeleteStreamRequest(scope=scope, stream=stream))

In [ ]:
#list(pravega_client.ListStreams(pravega.pb.ListStreamsRequest(scope=scope)))

In [ ]:
event = json.dumps(dict(size=int(.5*1024**3), timestamp=int(time.time()*1000))).encode('utf-8')
pravega_client.WriteEvents(iter([pravega.pb.WriteEventsRequest(scope=scope, stream=stream, event=event)]))

In [ ]:
stream_info = pravega_client.GetStreamInfo(pravega.pb.GetStreamInfoRequest(
            scope=scope,
            stream=stream,
        ))
#stream_info

In [ ]:
pravega_client.TruncateStream(pravega.pb.TruncateStreamRequest(scope=scope, stream=stream, stream_cut=stream_info.tail_stream_cut))

In [ ]:
list(pravega_client.ReadEvents(pravega.pb.ReadEventsRequest(scope=scope, stream=stream, from_stream_cut=None, to_stream_cut=stream_info.tail_stream_cut)))